# Custom Drone Navigation Environment

In [ ]:
import gym
import numpy as np
from gym import spaces
import pybullet as p
import pybullet_data

class CustomDroneEnv(gym.Env):
    def __init__(self):
        super(CustomDroneEnv, self).__init__()

        # Observation space: Position and velocity (x, y, z, vx, vy, vz)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

        # Action space: Changes to rotor speeds (assume 4 rotors)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(4,), dtype=np.float32)

        # PyBullet setup
        self.physics_client = p.connect(p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.8)

        self.drone = None
        self.start_pos = None
        self.target_pos = None
        self.episode_step = 0

        self._max_episode_steps = 200
        self.shaded_areas = []  # Dynamic obstacle zones

    def reset(self):
        # Reset episode state
        self.episode_step = 0
        p.resetSimulation()

        # Load ground plane
        p.loadURDF("plane.urdf")

        # Initialize random start and target positions
        self.start_pos = np.random.uniform(low=[-5, -5, 1], high=[5, 5, 2])
        self.target_pos = np.random.uniform(low=[-5, -5, 1], high=[5, 5, 2])

        while np.linalg.norm(self.start_pos - self.target_pos) < 2.0:  # Ensure target isn't too close
            self.target_pos = np.random.uniform(low=[-5, -5, 1], high=[5, 5, 2])

        # Spawn drone
        self.drone = p.loadURDF("quadrotor.urdf", basePosition=self.start_pos)

        # Generate dynamic shaded/obstacle areas
        self.shaded_areas = [np.random.uniform(-5, 5, size=(3,)) for _ in range(3)]

        return self._get_observation()

    def _get_observation(self):
        pos, vel = p.getBasePositionAndOrientation(self.drone)[0], p.getBaseVelocity(self.drone)[0]
        return np.array(list(pos) + list(vel))

    def step(self, action):
        # Apply action to control the drone
        for i, rotor_force in enumerate(action):
            p.applyExternalForce(
                self.drone, linkIndex=-1, forceObj=[0, 0, rotor_force * 10], posObj=[0, 0, 0], flags=p.WORLD_FRAME
            )

        p.stepSimulation()

        # Get updated observation
        obs = self._get_observation()
        current_pos = obs[:3]

        # Compute reward
        reward = -np.linalg.norm(self.target_pos - current_pos)  # Reward for getting closer to target
        if np.linalg.norm(self.target_pos - current_pos) < 0.5:  # Target reached
            reward += 100
            done = True
        elif self.episode_step >= self._max_episode_steps:
            done = True
        elif any(np.linalg.norm(current_pos - area) < 1.0 for area in self.shaded_areas):  # Collision penalty
            reward -= 50
            done = True
        else:
            done = False

        self.episode_step += 1

        return obs, reward, done, {}

    def render(self, mode="human"):
        pass

    def close(self):
        p.disconnect()
